# Aggregating and Combining `pandas` DataFrames

In [1]:
import pandas as pd
import numpy as np
import requests as rq
from sklearn.preprocessing import OneHotEncoder
from zipfile import ZipFile

## Learning Goals

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging and appending 

We'll work with the Austin Animal Center dataset and with data from King County's Department of Assessments (Seattle housing data).

### Austin Animal Center Data

In [2]:
data = rq.get('https://data.austintexas.gov/resource/9t4d-g238.json').text

In [3]:
animals = pd.read_json(data)

In [4]:
animals.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
0,A854124,Jett,2022-04-02 16:59:00,2022-04-02T16:59:00.000,2020-03-30T00:00:00.000,Rto-Adopt,Dog,Spayed Female,2 years,Staffordshire,Black/White,NaN
1,A854143,Chelsey,2022-04-02 16:40:00,2022-04-02T16:40:00.000,2021-03-30T00:00:00.000,Adoption,Dog,Spayed Female,1 year,Siberian Husky,Black/Cream,NaN
2,A854056,*Butterfinger,2022-04-02 16:28:00,2022-04-02T16:28:00.000,2022-01-29T00:00:00.000,Adoption,Dog,Spayed Female,2 months,Australian Shepherd Mix,Brown/White,Offsite
3,A853315,*Fanty,2022-04-02 16:19:00,2022-04-02T16:19:00.000,2021-11-01T00:00:00.000,Adoption,Dog,Spayed Female,4 months,Labrador Retriever Mix,Brown,Offsite
4,A840145,Echo,2022-04-02 16:14:00,2022-04-02T16:14:00.000,2021-02-03T00:00:00.000,Adoption,Dog,Neutered Male,1 year,Australian Shepherd/Australian Shepherd,Brown/White,NaN


In [5]:
animals.dtypes

animal_id                   object
name                        object
datetime            datetime64[ns]
monthyear                   object
date_of_birth               object
outcome_type                object
animal_type                 object
sex_upon_outcome            object
age_upon_outcome            object
breed                       object
color                       object
outcome_subtype             object
dtype: object

## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [6]:
animals.groupby('animal_type')

In [7]:
animals.columns

Index(['animal_id', 'name', 'datetime', 'monthyear', 'date_of_birth',
       'outcome_type', 'animal_type', 'sex_upon_outcome', 'age_upon_outcome',
       'breed', 'color', 'outcome_subtype'],
      dtype='object')

We can group by multiple columns, and also return a DataFrameGroupBy object

Notice the object type [DataFrameGroupBy](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) object. 

### `.groups` and `.get_group()`

In [8]:
animals.groupby(['animal_type', 'outcome_type'])

In [9]:
animals.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
0,A854124,Jett,2022-04-02 16:59:00,2022-04-02T16:59:00.000,2020-03-30T00:00:00.000,Rto-Adopt,Dog,Spayed Female,2 years,Staffordshire,Black/White,NaN
1,A854143,Chelsey,2022-04-02 16:40:00,2022-04-02T16:40:00.000,2021-03-30T00:00:00.000,Adoption,Dog,Spayed Female,1 year,Siberian Husky,Black/Cream,NaN
2,A854056,*Butterfinger,2022-04-02 16:28:00,2022-04-02T16:28:00.000,2022-01-29T00:00:00.000,Adoption,Dog,Spayed Female,2 months,Australian Shepherd Mix,Brown/White,Offsite
3,A853315,*Fanty,2022-04-02 16:19:00,2022-04-02T16:19:00.000,2021-11-01T00:00:00.000,Adoption,Dog,Spayed Female,4 months,Labrador Retriever Mix,Brown,Offsite
4,A840145,Echo,2022-04-02 16:14:00,2022-04-02T16:14:00.000,2021-02-03T00:00:00.000,Adoption,Dog,Neutered Male,1 year,Australian Shepherd/Australian Shepherd,Brown/White,NaN


In [10]:
# This retuns each group indexed by the group name: I.E. 'Bird', along with the row indices of each value
# .group tells me which row belongs to which animal

animals.groupby('animal_type').groups

{'Bird': [53, 772, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873], 'Cat': [18, 24, 25, 30, 31, 33, 34, 35, 43, 44, 45, 46, 47, 58, 62, 66, 67, 78, 82, 88, 93, 94, 112, 116, 117, 118, 119, 122, 125, 128, 136, 140, 141, 142, 148, 150, 154, 155, 156, 157, 170, 177, 178, 179, 180, 181, 189, 190, 194, 195, 198, 199, 202, 205, 206, 207, 208, 211, 213, 218, 222, 224, 235, 237, 239, 249, 253, 255, 256, 257, 258, 259, 260, 265, 266, 270, 271, 272, 273, 274, 275, 276, 282, 283, 284, 286, 287, 308, 318, 319, 321, 324, 325, 328, 329, 331, 332, 336, 339, 342, ...], 'Dog': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 19, 20, 21, 22, 26, 27, 28, 29, 32, 36, 37, 38, 39, 40, 41, 42, 50, 51, 52, 54, 55, 56, 57, 59, 60, 61, 63, 64, 65, 68, 73, 74, 75, 76, 80, 83, 84, 85, 86, 87, 89, 90, 91, 92, 95, 96, 97, 99, 100, 101, 106, 107, 108, 109, 110, 111, 113, 114, 115, 120, 121, 123, 124, 126, 127, 129, 132, 133, 134, 137, 138, 139, 143, 144, 145, 146, 147, 151, 152,

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is groups.

In [11]:
# Once we know the group indices, we can return the groups using those indices.
animals.groupby('animal_type').get_group('Dog')

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
0,A854124,Jett,2022-04-02 16:59:00,2022-04-02T16:59:00.000,2020-03-30T00:00:00.000,Rto-Adopt,Dog,Spayed Female,2 years,Staffordshire,Black/White,NaN
1,A854143,Chelsey,2022-04-02 16:40:00,2022-04-02T16:40:00.000,2021-03-30T00:00:00.000,Adoption,Dog,Spayed Female,1 year,Siberian Husky,Black/Cream,NaN
2,A854056,*Butterfinger,2022-04-02 16:28:00,2022-04-02T16:28:00.000,2022-01-29T00:00:00.000,Adoption,Dog,Spayed Female,2 months,Australian Shepherd Mix,Brown/White,Offsite
3,A853315,*Fanty,2022-04-02 16:19:00,2022-04-02T16:19:00.000,2021-11-01T00:00:00.000,Adoption,Dog,Spayed Female,4 months,Labrador Retriever Mix,Brown,Offsite
4,A840145,Echo,2022-04-02 16:14:00,2022-04-02T16:14:00.000,2021-02-03T00:00:00.000,Adoption,Dog,Neutered Male,1 year,Australian Shepherd/Australian Shepherd,Brown/White,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,A851555,*Squeak,2022-02-25 10:46:00,2022-02-25T10:46:00.000,2021-12-14T00:00:00.000,Transfer,Dog,Spayed Female,2 months,Chihuahua Shorthair Mix,Black/White,Out State
996,A851523,*Honeybee,2022-02-25 10:45:00,2022-02-25T10:45:00.000,2021-12-13T00:00:00.000,Transfer,Dog,Spayed Female,2 months,Labrador Retriever Mix,Yellow/White,Out State
997,A843515,Jax,2022-02-25 10:45:00,2022-02-25T10:45:00.000,2017-03-02T00:00:00.000,Transfer,Dog,Neutered Male,4 years,Great Pyrenees,White/Tan,Out State
998,A843516,Lucas,2022-02-25 10:44:00,2022-02-25T10:44:00.000,2017-03-02T00:00:00.000,Transfer,Dog,Neutered Male,4 years,Great Pyrenees,White,Out State


#### Multi-Indexing

In [12]:
# Same goes for multi index groupbys
animal_outcome = animals.groupby(['animal_type', 'outcome_type'])
animal_outcome.groups

{('Bird', 'Adoption'): [772, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873], ('Bird', 'Euthanasia'): [53], ('Cat', 'Adoption'): [24, 25, 30, 33, 34, 35, 43, 58, 62, 66, 88, 94, 118, 125, 148, 150, 154, 156, 157, 189, 190, 198, 199, 202, 205, 206, 207, 208, 218, 222, 224, 235, 237, 249, 253, 256, 265, 266, 270, 286, 287, 308, 318, 319, 321, 324, 325, 328, 329, 331, 332, 336, 339, 342, 346, 348, 351, 369, 385, 392, 393, 404, 412, 414, 427, 430, 434, 448, 450, 451, 454, 462, 468, 484, 485, 491, 499, 503, 505, 508, 514, 516, 521, 522, 523, 533, 534, 577, 579, 592, 594, 595, 596, 597, 601, 609, 614, 615, 618, 623, ...], ('Cat', 'Died'): [136, 903, 946], ('Cat', 'Disposal'): [112], ('Cat', 'Euthanasia'): [18, 47, 119, 122, 213, 366, 402, 418, 443, 470, 682, 790, 791, 792, 875, 942], ('Cat', 'Return to Owner'): [45, 155, 170, 211, 239, 467, 602, 698, 776, 857, 915], ('Cat', 'Rto-Adopt'): [31, 831], ('Cat', 'Transfer'): [44, 46, 67, 78, 82, 93, 116, 117, 128, 1

In [13]:
# animal_outcome.groups is a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

dict_keys([('Bird', 'Adoption'), ('Bird', 'Euthanasia'), ('Cat', 'Adoption'), ('Cat', 'Died'), ('Cat', 'Disposal'), ('Cat', 'Euthanasia'), ('Cat', 'Return to Owner'), ('Cat', 'Rto-Adopt'), ('Cat', 'Transfer'), ('Dog', 'Adoption'), ('Dog', 'Died'), ('Dog', 'Euthanasia'), ('Dog', 'Return to Owner'), ('Dog', 'Rto-Adopt'), ('Dog', 'Transfer'), ('Other', 'Adoption'), ('Other', 'Died'), ('Other', 'Disposal'), ('Other', 'Euthanasia'), ('Other', 'Return to Owner'), ('Other', 'Transfer')])

In [14]:
animal_outcome.groups.values()

dict_values([Int64Index([772, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869,
            870, 871, 872, 873],
           dtype='int64'), Int64Index([53], dtype='int64'), Int64Index([ 24,  25,  30,  33,  34,  35,  43,  58,  62,  66,
            ...
            950, 953, 957, 959, 967, 969, 970, 978, 980, 986],
           dtype='int64', length=155), Int64Index([136, 903, 946], dtype='int64'), Int64Index([112], dtype='int64'), Int64Index([18, 47, 119, 122, 213, 366, 402, 418, 443, 470, 682, 790, 791, 792,
            875, 942],
           dtype='int64'), Int64Index([45, 155, 170, 211, 239, 467, 602, 698, 776, 857, 915], dtype='int64'), Int64Index([31, 831], dtype='int64'), Int64Index([ 44,  46,  67,  78,  82,  93, 116, 117, 128, 140, 141, 142, 177,
            178, 179, 180, 181, 194, 195, 255, 257, 258, 259, 260, 271, 272,
            273, 274, 275, 276, 282, 283, 284, 357, 358, 361, 362, 363, 364,
            365, 387, 388, 424, 465, 466, 496, 497, 528, 537, 561, 562, 563,


In [15]:
# We can then get a specific group, such as Cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
24,A854129,*Shawn,2022-04-01 18:44:00,2022-04-01T18:44:00.000,2021-10-30T00:00:00.000,Adoption,Cat,Neutered Male,5 months,Domestic Shorthair Mix,Brown Tabby/White,NaN
25,A854127,*Cory,2022-04-01 18:22:00,2022-04-01T18:22:00.000,2021-10-30T00:00:00.000,Adoption,Cat,Neutered Male,5 months,Siamese,Lynx Point,NaN
30,A853978,NaN,2022-04-01 17:41:00,2022-04-01T17:41:00.000,2021-09-28T00:00:00.000,Adoption,Cat,Spayed Female,6 months,Domestic Shorthair,Calico,NaN
33,A853964,A853964,2022-04-01 16:57:00,2022-04-01T16:57:00.000,2021-11-13T00:00:00.000,Adoption,Cat,Spayed Female,4 months,Domestic Shorthair Mix,Torbie,NaN
34,A853977,NaN,2022-04-01 16:19:00,2022-04-01T16:19:00.000,2021-10-28T00:00:00.000,Adoption,Cat,Spayed Female,5 months,Domestic Shorthair,Calico,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
969,A846497,*Hugs,2022-02-26 07:49:00,2022-02-26T07:49:00.000,2021-10-18T00:00:00.000,Adoption,Cat,Spayed Female,4 months,Domestic Shorthair,Torbie,Foster
970,A846499,*Kisses,2022-02-26 07:49:00,2022-02-26T07:49:00.000,2021-10-18T00:00:00.000,Adoption,Cat,Spayed Female,4 months,Domestic Shorthair,Calico,Foster
978,A851058,Stuart,2022-02-25 17:01:00,2022-02-25T17:01:00.000,2018-02-05T00:00:00.000,Adoption,Cat,Spayed Female,4 years,Domestic Shorthair,Black/White,NaN
980,A851957,Ruby,2022-02-25 16:22:00,2022-02-25T16:22:00.000,2021-10-22T00:00:00.000,Adoption,Cat,Spayed Female,4 months,Domestic Shorthair,Brown Tabby,NaN


### Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [16]:
animals.groupby('animal_type').count()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
animal_type,,,,,,,,,,,
Bird,18,1,18,18,18,18,18,18,18,18,1
Cat,267,230,267,267,267,267,267,267,267,267,120
Dog,632,592,632,632,632,632,632,632,632,632,246
Other,83,24,83,83,83,83,83,83,83,83,75


### Exercise

Use `.groupby()` to find the most recently born of each (main) animal type.

In [17]:
animals.groupby('animal_type')['date_of_birth'].max()

animal_type
Bird     2020-03-28T00:00:00.000
Cat      2022-03-31T00:00:00.000
Dog      2022-03-29T00:00:00.000
Other    2021-07-02T00:00:00.000
Name: date_of_birth, dtype: object

<details>
    <summary>Answer</summary>
    <code>animals.groupby('animal_type')['date_of_birth'].max()</code>
    </details>

## Pivoting a DataFrame

### `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [18]:
animals.groupby(by=['outcome_type', 'sex_upon_outcome']).agg(len)

animal_id  name  datetime  monthyear  \
outcome_type    sex_upon_outcome                                         
Adoption        Intact Female            24    24        24         24   
                Intact Male               5     5         5          5   
                Neutered Male           250   250       250        250   
                Spayed Female           245   245       245        245   
Died            Intact Female             1     1         1          1   
                Intact Male               2     2         2          2   
                Neutered Male             3     3         3          3   
                Spayed Female             1     1         1          1   
                Unknown                   2     2         2          2   
Disposal        Unknown                   3     3         3          3   
Euthanasia      Intact Female             7     7         7          7   
                Intact Male              11    11        11         11   
                Neutered Male             5     5         5          5   
                Spayed Female             1     1         1          1   
                Unknown                  63    63        63         63   
Return to Owner Intact Female            30    30        30         30   
                Intact Male              34    34        34         34   
                Neutered Male            34    34        34         34   
                Spayed Female            23    23        23         23   
                Unknown                   3     3         3          3   
Rto-Adopt       Intact Female             1     1         1          1   
                Neutered Male             8     8         8          8   
                Spayed Female             7     7         7          7   
Transfer        Intact Female            69    69        69         69   
                Intact Male              66    66        66         66   
                Neutered Male            59    59        59         59   
                Spayed Female            35    35        35         35   
                Unknown                   8     8         8          8   

                                  date_of_birth  animal_type  \
outcome_type    sex_upon_outcome                               
Adoption        Intact Female                24           24   
                Intact Male                   5            5   
                Neutered Male               250          250   
                Spayed Female               245          245   
Died            Intact Female                 1            1   
                Intact Male                   2            2   
                Neutered Male                 3            3   
                Spayed Female                 1            1   
                Unknown                       2            2   
Disposal        Unknown                       3            3   
Euthanasia      Intact Female                 7            7   
                Intact Male                  11           11   
                Neutered Male                 5            5   
                Spayed Female                 1            1   
                Unknown                      63           63   
Return to Owner Intact Female                30           30   
                Intact Male                  34           34   
                Neutered Male                34           34   
                Spayed Female                23           23   
                Unknown                       3            3   
Rto-Adopt       Intact Female                 1            1   
                Neutered Male                 8            8   
                Spayed Female                 7            7   
Transfer        Intact Female                69           69   
                Intact Male                  66           66   
                Neutered Male                59           59   
                Spaye

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

#### Example

In [19]:
df = pd.DataFrame({"sex": ["male", "male", "male", "male", "male",
                          "female", "female", "female", "female"],
                    "num_puppies": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "breed": ["terrier", "retriever", "retriever", "terrier",
                          "terrier", "retriever", "terrier", "terrier",
                          "retriever"],
                    "past_owners": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "family_members": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,sex,num_puppies,breed,past_owners,family_members
0,male,one,terrier,1,2
1,male,one,retriever,2,4
2,male,one,retriever,2,5
3,male,two,terrier,3,5
4,male,two,terrier,3,6
5,female,one,retriever,4,6
6,female,one,terrier,5,8
7,female,two,terrier,6,9
8,female,two,retriever,7,9


In [20]:
# This first example aggregates values by taking the sum.

table = pd.pivot_table(df, values='past_owners', index=['sex', 'num_puppies'],
                     columns=['breed'], aggfunc=np.sum)
table

breed               retriever  terrier
sex    num_puppies                    
female one                4.0      5.0
       two                7.0      6.0
male   one                4.0      1.0
       two                NaN      6.0

In [21]:
table.index

MultiIndex([('female', 'one'),
            ('female', 'two'),
            (  'male', 'one'),
            (  'male', 'two')],
           names=['sex', 'num_puppies'])

In [22]:
table.reset_index() # flatten

breed,sex,num_puppies,retriever,terrier
0,female,one,4.0,5.0
1,female,two,7.0,6.0
2,male,one,4.0,1.0
3,male,two,NaN,6.0


#### Back to Austin animals

In [23]:
animals.pivot_table(index='outcome_type', columns='sex_upon_outcome', aggfunc=len)

age_upon_outcome                                          \
sex_upon_outcome    Intact Female Intact Male Neutered Male Spayed Female   
outcome_type                                                                
Adoption                     24.0         5.0         250.0         245.0   
Died                          1.0         2.0           3.0           1.0   
Disposal                      NaN         NaN           NaN           NaN   
Euthanasia                    7.0        11.0           5.0           1.0   
Return to Owner              30.0        34.0          34.0          23.0   
Rto-Adopt                     1.0         NaN           8.0           7.0   
Transfer                     69.0        66.0          59.0          35.0   

                             animal_id                            \
sex_upon_outcome Unknown Intact Female Intact Male Neutered Male   
outcome_type                                                       
Adoption             NaN          24.0         5.0         250.0   
Died                 2.0           1.0         2.0           3.0   
Disposal             3.0           NaN         NaN           NaN   
Euthanasia          63.0           7.0        11.0           5.0   
Return to Owner      3.0          30.0        34.0          34.0   
Rto-Adopt            NaN           1.0         NaN           8.0   
Transfer             8.0          69.0        66.0          59.0   

                                        ...          name              \
sex_upon_outcome Spayed Female Unknown  ... Intact Female Intact Male   
outcome_type                            ...                             
Adoption                 245.0     NaN  ...          24.0         5.0   
Died                       1.0     2.0  ...           1.0         2.0   
Disposal                   NaN     3.0  ...           NaN         NaN   
Euthanasia                 1.0    63.0  ...           7.0        11.0   
Return to Owner           23.0     3.0  ...          30.0        34.0   
Rto-Adopt                  7.0     NaN  ...           1.0         NaN   
Transfer                  35.0     8.0  ...          69.0        66.0   

                                                     outcome_subtype  \
sex_upon_outcome Neutered Male Spayed Female Unknown   Intact Female   
outcome_type                                                           
Adoption                 250.0         245.0     NaN            24.0   
Died                       3.0           1.0     2.0             1.0   
Disposal                   NaN           NaN     3.0             NaN   
Euthanasia                 5.0           1.0    63.0             7.0   
Return to Owner           34.0          23.0     3.0            30.0   
Rto-Adopt                  8.0           7.0     NaN             1.0   
Transfer                  59.0          35.0     8.0            69.0   

                                                                  
sex_upon_outcome Intact Male Neutered Male Spayed Female Unknown  
outcome_type                                                      
Adoption                 5.0         250.0         245.0     NaN  
Died                     2.0           3.0           1.0     2.0  
Disposal                 NaN           NaN           NaN     3.0  
Euthanasia              11.0           5.0           1.0    63.0  
Return to Owner         34.0          34.0          23.0     3.0  
Rto-Adopt                NaN           8.0           7.0     NaN  
Transfer                66.0          59.0          35.0     8.0  

[7 rows x 50 columns]

### Exercise

Use `.pivot_table()` to add up the number of my tasks by category. Hint: Use `sum()` as your aggregating function.

In [24]:
tasks = pd.DataFrame({'category': ['house', 'house', 'school', 'school'],
                      'descr': ['kitchen', 'laundry', 'git', 'Python'],
                      'priority': [2, 3, 4, 1], 'num_tasks': [2, 1, 2, 3]})

tasks

,category,descr,priority,num_tasks
0,house,kitchen,2,2
1,house,laundry,3,1
2,school,git,4,2
3,school,Python,1,3


In [25]:
tasks.pivot_table(index='category', values='num_tasks', aggfunc=sum)

,num_tasks
category,
house,3
school,5


<details>
    <summary>Answer</summary>
    <code>tasks.pivot_table(values='num_tasks', index='category', aggfunc=sum)</code>
    </details>

## Methods for Combining DataFrames: `.join()`, `.merge()`, `pd.concat()`

### `.join()`

In [26]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [27]:
toy2

,age,MP
0,63,100
1,33,200


In [28]:
# We can't just join these as they are, since we haven't specified our suffixes.

# toy1.join(toy2)

In [29]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [30]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


For more on this method, check out the [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)!

### `.merge()`

Or we could use `.merge()`:

In [31]:
toy1.merge(toy2)

,age,HP,MP
0,63,142,100
1,33,47,200


In [32]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [33]:
states = pd.read_csv('data/states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


### The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

In [34]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [35]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,WA,evergreen,Olympia
1,miles,200.0,WA,WA,evergreen,Olympia
2,alan,170.0,TX,TX,alamo,Austin
3,rachel,200.0,TX,TX,alamo,Austin
4,alison,300.0,DC,DC,district,Washington
5,NaN,NaN,NaN,OH,buckeye,Columbus
6,NaN,NaN,NaN,OR,beaver,Salem


### `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [36]:
ds_full = pd.concat([ds_chars, states])
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,NaN,NaN,NaN
1,miles,200.0,WA,NaN,NaN,NaN
2,alan,170.0,TX,NaN,NaN,NaN
3,alison,300.0,DC,NaN,NaN,NaN
4,rachel,200.0,TX,NaN,NaN,NaN
0,NaN,NaN,NaN,WA,evergreen,Olympia
1,NaN,NaN,NaN,TX,alamo,Austin
2,NaN,NaN,NaN,DC,district,Washington
3,NaN,NaN,NaN,OH,buckeye,Columbus
4,NaN,NaN,NaN,OR,beaver,Salem


`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [37]:
ds_full = pd.concat([ds_chars, states], axis=1)
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,TX,alamo,Austin
2,alan,170,TX,DC,district,Washington
3,alison,300,DC,OH,buckeye,Columbus
4,rachel,200,TX,OR,beaver,Salem


## King County Assessments

As data scientists, we want to build a model to predict the sale price of a house in Seattle in 2019, based on its square footage. We know that the King County Department of Assessments has comprehensive data available on real property sales in the Seattle area. We need to prepare the data.

### First, get the data!

Go [here](https://info.kingcounty.gov/assessor/DataDownload/default.aspx) and download two files: "Real Property Sales" and "Residential Building". Then unzip them. (Or you can run the cells below if you prefer.)

In [38]:
# %%bash
# cd data
# curl -o property_sales.zip https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip 

In [39]:
# %%bash
# cd data
# curl -o res_bldg.zip https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip 

In [40]:
# zf = ZipFile('data/property_sales.zip', 'r')
# zf.extractall('data')
# zf.close()

In [41]:
# zf = ZipFile('data/res_bldg.zip', 'r')
# zf.extractall('data')
# zf.close()

In [42]:
# You'll need to use a new encoding here. List of all encodings here:
# https://docs.python.org/3/library/codecs.html#standard-encodings

# Both of these csv files have many columns, so we'll just pre-select
# which ones we want to use.

sales_df = pd.read_csv('data/EXTR_RPSale.csv',
                       encoding='latin-1',
                       usecols=['Major', 'Minor', 'DocumentDate', 'SalePrice'])

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [43]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220587 entries, 0 to 2220586
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Major         object
 1   Minor         object
 2   DocumentDate  object
 3   SalePrice     int64 
dtypes: int64(1), object(3)
memory usage: 67.8+ MB


In [44]:
bldg_df = pd.read_csv('data/EXTR_ResBldg.csv',
                     usecols=['Major', 'Minor', 'SqFtTotLiving', 'ZipCode'])

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
bldg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522030 entries, 0 to 522029
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Major          522030 non-null  int64 
 1   Minor          522030 non-null  int64 
 2   ZipCode        470868 non-null  object
 3   SqFtTotLiving  522030 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 15.9+ MB


In [46]:
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

In [47]:
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560


In [48]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1544897 entries, 0 to 1544896
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Major          1544897 non-null  object
 1   Minor          1544897 non-null  object
 2   DocumentDate   1544897 non-null  object
 3   SalePrice      1544897 non-null  int64 
 4   ZipCode        1404584 non-null  object
 5   SqFtTotLiving  1544897 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 82.5+ MB


We can see right away that we're missing ZIP codes for many of the sales transactions.

In [49]:
sales_data.loc[sales_data['ZipCode'].isna()].head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
28,226700,160,05/08/2003,0,NaN,1560
29,226700,160,05/11/1996,0,NaN,1560
30,226700,160,09/08/2011,0,NaN,1560
31,226700,160,10/09/2018,855000,NaN,1560
32,226700,160,01/24/2020,0,NaN,1560


### Exercise

What percentage of housing records are missing ZIP codes?

In [50]:
sales_data['ZipCode'].isna().sum() / len(sales_data)

0.09082353063019735

<details>
    <summary>Answer</summary>
    <code>sales_data['ZipCode'].isna().sum() / sales_data.shape[0]</code>
    </details>

Let's drop the rows with missing zip codes.

In [51]:
sales_data = sales_data.dropna(subset=['ZipCode'], how='all')
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560


In [52]:
sales_data.isna().sum()

Major            0
Minor            0
DocumentDate     0
SalePrice        0
ZipCode          0
SqFtTotLiving    0
dtype: int64

In [53]:
sales_data = sales_data.loc[~sales_data['ZipCode'].isna(), :]
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560


## Time Permitting: Data Cleaning with Pandas

### 1. Investigate and drop rows with invalid values in the SalePrice and SqFtTotLiving columns.

In [54]:
sales_data.describe()

,SalePrice,SqFtTotLiving
count,1.404584e+06,1.404584e+06
mean,3.089713e+05,2.109228e+03
std,7.589833e+05,9.776886e+02
min,-4.000000e+02,0.000000e+00
25%,0.000000e+00,1.440000e+03
50%,1.675000e+05,1.940000e+03
75%,3.800000e+05,2.570000e+03
max,3.750000e+07,4.816000e+04


In [55]:
sales_data['SalePrice'].min()

-400

In [56]:
sales_data = sales_data[sales_data['SalePrice'] > 10000]

<details>
    <summary>One possible answer here</summary>
    <code>sales_data = sales_data[sales_data['SalePrice'] > 10000]</code>
    </details>

### 2. Investigate and handle non-numeric ZipCode values

Can you find a way to shorten ZIP+4 codes to the first five digits?

In [57]:
sales_data['ZipCode']

0            98168
1            98168
3            98168
4            98168
7            98133
            ...   
1544890      98070
1544891    98103.0
1544893      98007
1544895    98033.0
1544896      98007
Name: ZipCode, Length: 914795, dtype: object

In [58]:
# There are some values which could not be converted into integer. 
# These values below are formatted wrong, using split() and strip() fix them first.

def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].unique()

array(['98199-3014', '98028-8908', '98042-3001', '98031-3173',
       '98033-4917', '98058-9018', '98177-4810', '98058-7983',
       '98136-1728', '98052-1963', '98074-6315', '98042-4811',
       '98038-3112', '98023-2712', '98074-3438', '98028-4768',
       '98034-2540', '98074-4417', '98059-7428', '# 3', '98023-7330',
       '98006-3954', '98074-9301', '98028-2118', '98053-5932',
       '98074-2814', '98075-6010', '98075-9644', '98118-6116',
       '98074-5411', ' ', '98028-4377', '98198-8930', '98075-9645',
       '98045-9504', '98028-7915', '98107-5353', '98028-4228',
       '98074-7242', '98103-8173', '98075-9158', '98074-3908',
       '98033-8239', '98028-4712', '98118-3167', '98029-3602',
       '98028-4505', '98075-9517', '98028-1654', '98075-8010',
       '98074-6568', '98058-5344', '98011-3777', '98023-7841',
       '98059-5449', '98074-3738', 'D1 south', '98074-4092', '98042-8114',
       '98146-3008', '98059-7120', '98028-6100', 'WA', '98042-8215',
       '98028-4625', '980

In [59]:
'98199-3014'.split('-')[0][0:5]

'98199'

In [60]:
sales_data.drop(sales_data[sales_data['ZipCode'] == 'WA'].index, axis=0, inplace=True)
sales_data.drop(sales_data[sales_data['ZipCode'] == 'D1 south'].index, axis=0, inplace=True)
sales_data.drop(sales_data[sales_data['ZipCode'] == ' '].index, axis=0, inplace=True)
sales_data.drop(sales_data[sales_data['ZipCode'] == '# 3'].index, axis=0, inplace=True)
sales_data.drop(sales_data[sales_data['ZipCode'] == ', '].index, axis=0, inplace=True)

In [61]:
sales_data['ZipCode']= sales_data['ZipCode'].apply(lambda x: str(x)) # sales_data['ZipCode'].astype(str) does NOT work WHY?
sales_data['ZipCode']= sales_data['ZipCode'].apply(lambda x: x.split("-")[0][0:5])
sales_data['ZipCode']= sales_data['ZipCode'].apply(lambda x: int(x))
sales_data['ZipCode']

0          98168
1          98168
3          98168
4          98168
7          98133
           ...  
1544890    98070
1544891    98103
1544893    98007
1544895    98033
1544896    98007
Name: ZipCode, Length: 914787, dtype: int64

**An alternative is to use these two codes below:**

In [62]:
# Check again to make sure there are no wrong values

def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].unique()

array([], dtype=int64)

In [63]:
def five_digit_ZIP(x):
    try:
        return int(str(x)[:5])
    except:
        return x
sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
sales_data = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == True, :]
sales_data['ZipCode'] = sales_data['ZipCode'].map(int)

<details>
    <summary>One possible answer here</summary>
    <code>def five_digit_ZIP(x):
    try:
        return int(str(x)[:5])
    except:
        return x
sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
sales_data = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == True, :]
sales_data['ZipCode'] = sales_data['ZipCode'].map(int)</code>
    </details>

### 3. Add a column for PricePerSqFt



In [64]:
sales_data['PricePerSqFt'] =  sales_data['SalePrice']/sales_data['SqFtTotLiving']
sales_data['PricePerSqFt'].head()

0     66.346154
1    142.243590
3    126.923077
4    236.538462
7    217.415730
Name: PricePerSqFt, dtype: float64

<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'] = sales_data['SalePrice'] / sales_data['SqFtTotLiving']</code>
    </details>

### 4. Subset the data to 2021 sales only.

We can assume that the DocumentDate is approximately the sale date.

In [65]:
sales_data ['DocumentDate'].dtype

dtype('O')

In [66]:
sales_data['DocumentDate'] = pd.to_datetime(sales_data['DocumentDate'])

In [67]:
sales_data ['DocumentDate'].sample(2)

1267065   1993-09-30
317475    2020-07-28
Name: DocumentDate, dtype: datetime64[ns]

sales_data = sales_data[sales_data['DocumentDate'] >= '2021-01-01']
sales_data = sales_data[sales_data['DocumentDate'] < '2022-01-01']
sales_data.head()

In [68]:
sales_data = sales_data[(sales_data['DocumentDate'] >= '2021-01-01') & (sales_data['DocumentDate'] < '2022-01-01')]
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving,PricePerSqFt
38,891050,230,2021-12-13,920000,98133,2770,332.129964
46,118000,275,2021-09-29,311000,98178,2880,107.986111
147,923890,1045,2021-12-17,726000,98136,1230,590.243902
174,251701,640,2021-03-22,605000,98042,2290,264.192140
222,329370,160,2021-03-24,815000,98133,2500,326.000000


<details>
    <summary>Answer here</summary>
    <code>sales_data['DocumentDate'] = pd.to_datetime(sales_data['DocumentDate'])
sales_data = sales_data.loc[sales_data['DocumentDate'] > '12/31/2020']</code>
    </details>

### 5. What is the mean price per square foot for a house sold in Seattle in 2021?

In [69]:
sales_data['PricePerSqFt'].mean()

513.5327602840078

<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'].mean()</code>
    </details>

## Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

### Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [70]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

,0,1
0,1.000000e+09,2.000000e+09
1,3.000000e+09,4.000000e+09


Then we can use:

In [71]:
pd.set_option('display.float_format', '{:.2f}'.format)
df

,0,1
0,1000000000.00,2000000000.00
1,3000000000.00,4000000000.00


### See More Rows

Or suppose we want `pandas` to show more rows.

In [72]:
df2 = pd.DataFrame(np.array(range(100)))
df2

,0
0,0
1,1
2,2
3,3
4,4
...,...
95,95
96,96
97,97
98,98


In that case we can use:

In [73]:
pd.set_option('display.max_rows', 100)

df2

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).